In [ ]:
# parser for generating mcnp files

# mcnp file stucture:
# 1. title card
# 2. cell cards
# 3. surface cards
# 4. data cards
# 5. material cards
# 6. tally cards
# 7. plot cards
# 8. output cards

# cell card structure:
# c i j k l
# c = cell number
# i = material number
# j = density
# k = surface number
# l = importance

# surface card structure:
# s i j k
# s = surface number
# i = surface type
# j = surface parameter 1
# k = surface parameter 2
# surface types:
# 1 = plane
# 2 = sphere
# 3 = cylinder
# 4 = cone
# 5 = ellipsoid
# 6 = torus
# 7 = hexagonal prism
# 8 = rectangular prism
# 9 = hexagonal cylinder
# 10 = rectangular cylinder
# 11 = rectangular parallelepiped
# 12 = elliptical cylinder

# material card structure:
# m i
# m = material card number
# i = material number

In [2]:
import pandas as pd
import numpy as np

In [3]:
class Material:
    def __init__(self, name, id, composition, hydration=None):
        self.name = name
        self.id = id
        self.composition = composition
        self.hydration = hydration
    def __repr__(self):
        return f"Material({self.name}, {self.id}, {self.composition}, {self.hydration})"
    def __str__(self):
        mats = ' '.join([f"{key} {value}\n" for key, value in self.composition.items()])
        return f"m{self.id} {mats}"
    def __getitem__(self, key):
        return self.composition[key]
    def __setitem__(self, key, value):
        self.composition[key] = value
    def __iter__(self):
        return iter(self.composition)
    def __len__(self):
        return len(self.composition)
    def __contains__(self, item):
        return item in self.composition
    def __eq__(self, other):
        return self.id == other.id
    def __ne__(self, other):
        return self.id != other.id
    def __lt__(self, other):
        return self.id < other.id
    def __le__(self, other):
        return self.id <= other.id
    def __gt__(self, other):
        return self.id > other.id
    def __ge__(self, other):
        return self.id >= other.id
    def __hash__(self):
        return hash(self.id)
    
    def to_dict(self):
        return {
            "name": self.name,
            "id": self.id,
            "hydration": self.hydration,
            "composition": self.composition
        }
    
    @classmethod

    def from_dict(cls, data):
        return cls(data["name"], data["id"], data["composition"], data["hydration"])
    
    @classmethod
    def from_series(cls, series):
        return cls(series["name"], series["id"], series["composition"], series["hydration"])
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
    
    @classmethod
    def to_dataframe(cls, materials):
        return pd.DataFrame([m.to_dict() for m in materials]).set_index("id")
    
    @classmethod
    def to_json(cls, materials):
        return cls.to_dataframe(materials).to_json(orient="records")
    
    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))
    
    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, materials, filename):
        cls.to_dataframe(materials).to_csv(filename)


In [5]:
mat = Material(
    "water", 
    1, 
    {1001: 2, 8016: 1})

'm1 1001 2\n 8016 1\n'

In [9]:
class surface():
    def __init__(self, surface_id, shape, params, group=None):
        self.surface_id = surface_id
        self.group = group
        self.shape = shape
        self.params = params
        self.position = params["pos"]
    def __repr__(self):
        return f"surface({self.surface_id}, {self.group}, {self.shape}, {self.params})"
    def __str__(self):
        return f"surface {self.surface_id}"
    def __getitem__(self, key):
        return self.params[key]
    def __setitem__(self, key, value):
        self.params[key] = value
    def __iter__(self):
        return iter(self.params)
    def __len__(self):
        return len(self.params)
    def __contains__(self, item):
        return item in self.params
    def __eq__(self, other):
        return self.surface_id == other.surface_id
    def __ne__(self, other):
        return self.surface_id != other.surface_id
    def __lt__(self, other):
        return self.surface_id < other.surface_id
    def __le__(self, other):
        return self.surface_id <= other.surface_id
    def __gt__(self, other):
        return self.surface_id > other.surface_id
    def __ge__(self, other):
        return self.surface_id >= other.surface_id
    def __hash__(self):
        return hash(self.surface_id)
    
    def to_dict(self):
        return {
            "surface_id": self.surface_id,
            "group": self.group,
            "shape": self.shape,
            "params": self.params
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(data["surface_id"], data["group"], data["shape"], data["params"])
    
    @classmethod
    def from_series(cls, series):
        return cls(series["surface_id"], series["group"], series["shape"], series["params"])
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
    
    @classmethod
    def to_dataframe(cls, surfaces):
        return pd.DataFrame([s.to_dict() for s in surfaces]).set_index("surface_id")
    
    @classmethod
    def to_json(cls, surfaces):
        return cls.to_dataframe(surfaces).to_json(orient="records")
    
    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))
    
    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, surfaces, filename):
        cls.to_dataframe(surfaces).to_csv(filename)


In [47]:
class sphere(surface):
    def __init__(self, surface_id, pos, r, group=None):
        self.pos = pos
        self.r = r
        params = {
            "pos": pos,
            "r": r
        }
        super().__init__(surface_id, "sphere", params, group)
    def __repr__(self):
        return f"sphere({self.surface_id}, {self.group}, {self.params})"
    def __str__(self):
        return f"sph {' '.join([str(p) for p in self.pos])} {self.r}"

In [48]:
ball = sphere(1, [0, 0, 0], 10)
str(ball)

'sph 0 0 0 10'

In [34]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return (v / norm).tolist()

In [57]:
class cylinder(surface):
    def __init__(self, surface_id, pos, r, height, dir, group=None):
        self.pos = pos
        self.r = r
        self.height = height
        self.dir = normalize(dir)
        self.group = group
        self.params = {
            "pos": pos,
            "r": r,
            "height": height,
            "dir": dir
        }
        super().__init__(surface_id, "cylinder", self.params, group)
    def __repr__(self):
        return f"cylinder({self.surface_id}, {' '.join([str(p)+'='+str(self.params[p]) for p in self.params.keys()])}"
    def __str__(self):
        
        return f"rcc {' '.join([str(p) for p in self.pos])} {' '.join([str(d*self.height) for d in self.dir])} {self.r}\n"

In [61]:
cyl = cylinder(2, [0, 0, 0], 10, 20, [0, 1, -1])
str(cyl)

'rcc 0 0 0 0.0 14.14213562373095 -14.14213562373095 10\n'

In [65]:
class box(surface):
    def __init__(self, surface_id, pos, l, w, h, group=None):
        self.pos = pos
        self.l = l #x
        self.w = w #y
        self.h = h #z
        params = {
            "pos": pos,
            "l": l,
            "w": w,
            "h": h
        }
        super().__init__(surface_id, "box", params, group)
    def __repr__(self):
        return f"box({self.surface_id}, {self.group}, {self.params})"
    def __str__(self):
        corner = [p - l/2 for p, l in zip(self.pos, [self.l, self.w, self.h])]
        return f"box {' '.join([str(p) for p in corner])} {self.l} 0 0 0 {self.w} 0 0 0 {self.h}\n"

In [72]:

cube = box(3, [0, 0, 0], 10, 10, 10)
str(cube) 

'box -5.0 -5.0 -5.0 10 0 0 0 10 0 0 0 10\n'

In [73]:
class cell():
    def __init__(self, cell_id, material, density, surfaces, importance=None):
        self.cell_id = cell_id
        self.material = material
        self.density = density
        self.surfaces = surfaces
        self.importance = importance
    def __repr__(self):
        return f"cell({self.cell_id}, {self.material}, {self.density}, {self.surfaces}, {self.importance})"
    def __str__(self):
        return f"{self.cell_id} {self.material} {self.density} {' '.join([str(surface) for surface in self.surfaces])} {self.importance}\n"
    def __getitem__(self, key):
        return self.surfaces[key]
    def __setitem__(self, key, value):
        self.surfaces[key] = value
    def __iter__(self):
        return iter(self.surfaces)
    def __len__(self):
        return len(self.surfaces)
    def __contains__(self, item):
        return item in self.surfaces
    def __eq__(self, other):
        return self.cell_id == other.cell_id
    def __ne__(self, other):
        return self.cell_id != other.cell_id
    def __lt__(self, other):
        return self.cell_id < other.cell_id
    def __le__(self, other):
        return self.cell_id <= other.cell_id
    def __gt__(self, other):
        return self.cell_id > other.cell_id
    def __ge__(self, other):
        return self.cell_id >= other.cell_id
    def __hash__(self):
        return hash(self.cell_id)
    
    def to_dict(self):
        return {
            "cell_id": self.cell_id,
            "material": self.material,
            "density": self.density,
            "surfaces": self.surfaces,
            "importance": self.importance
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(data["cell_id"], data["material"], data["density"], data["surfaces"], data["importance"])
    
    @classmethod
    def from_series(cls, series):
        return cls(series["cell_id"], series["material"], series["density"], series["surfaces"], series["importance"])
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
        
    @classmethod
    def to_dataframe(cls, cells):
        return pd.DataFrame([c.to_dict() for c in cells]).set_index("cell_id")
    
    @classmethod
    def to_json(cls, cells):
        return cls.to_dataframe(cells).to_json(orient="records")
    
    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))
    
    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, cells, filename):
        cls.to_dataframe(cells).to_csv(filename)

In [74]:
str(cell(1, 1, 1, [1, 2, 3], 1))

'1 1 1 1 2 3 1\n'

In [79]:
class source():
    def __init__(self, source_id, pos, energy):
        self.source_id = source_id
        self.pos = pos
        self.energy = energy
    def __repr__(self):
        return f"source({self.source_id}, {self.pos}, {self.energy})"
    def __str__(self):
        return f"sdef pos={' '.join([str(i) for i in self.pos])} erg={self.energy}\n"
    def __eq__(self, other):
        return self.source_id == other.source_id
    def __ne__(self, other):
        return self.source_id != other.source_id
    def __lt__(self, other):
        return self.source_id < other.source_id
    def __le__(self, other):
        return self.source_id <= other.source_id
    def __gt__(self, other):
        return self.source_id > other.source_id
    def __ge__(self, other):
        return self.source_id >= other.source_id
    def __hash__(self):
        return hash(self.source_id)
    
    def to_dict(self):
        return {
            "source_id": self.source_id,
            "pos": self.pos,
            "energy": self.energy
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(data["source_id"], data["pos"], data["energy"])
    
    @classmethod
    def from_series(cls, series):
        return cls(series["source_id"], series["pos"], series["energy"])
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
    
    @classmethod
    def to_dataframe(cls, sources):
        return pd.DataFrame([s.to_dict() for s in sources]).set_index("source_id")
    
    @classmethod
    def to_json(cls, sources):
        return cls.to_dataframe(sources).to_json(orient="records")
    

    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))
    
    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, sources, filename):
        cls.to_dataframe(sources).to_csv(filename)

In [80]:
sou = source(1, [0, 0, 0], 14)
str(sou)

'sdef pos=0 0 0 erg=14\n'

In [9]:
class tally():
    def __init__(self, tally_id, tally_type, particle, cells, energy_bins=None, time_bins=None, energy_range=None, time_range=None):
        self.tally_id = tally_id
        self.tally_type = tally_type
        self.particle = particle
        self.cells = cells
        self.energy_bins = energy_bins
        self.time_bins = time_bins
        self.energy_range = energy_range
        self.time_range = time_range

    def __repr__(self):
        return f"tally({self.tally_id}, {self.tally_type}, {self.particle}, {self.cells}, {self.energy_bins}, {self.time_bins}, {self.energy_range}, {self.time_range})"
    def __str__(self):
        return f"tally {self.tally_id}"
    def __getitem__(self, key):
        return self.cells[key]
    def __setitem__(self, key, value):
        self.cells[key] = value
    def __iter__(self):
        return iter(self.cells)
    def __len__(self):
        return len(self.cells)
    def __contains__(self, item):
        return item in self.cells
    def __eq__(self, other):
        return self.tally_id == other.tally_id
    def __ne__(self, other):
        return self.tally_id != other.tally_id
    def __lt__(self, other):
        return self.tally_id < other.tally_id
    def __le__(self, other):
        return self.tally_id <= other.tally_id
    def __gt__(self, other):
        return self.tally_id > other.tally_id
    def __ge__(self, other):
        return self.tally_id >= other.tally_id
    def __hash__(self):
        return hash(self.tally_id)
    
    def to_dict(self):
        return {
            "tally_id": self.tally_id,
            "tally_type": self.tally_type,
            "particle": self.particle,
            "cells": self.cells,
            "energy_bins": self.energy_bins,
            "time_bins": self.time_bins,
            "energy_range": self.energy_range,
            "time_range": self.time_range
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(data["tally_id"], data["tally_type"], data["particle"], data["cells"], data["energy_bins"], data["time_bins"], data["energy_range"], data["time_range"])
    
    @classmethod
    def from_series(cls, series):
        return cls(series["tally_id"], series["tally_type"], series["particle"], series["cells"], series["energy_bins"], series["time_bins"], series["energy_range"], series["time_range"])
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
    
    @classmethod
    def to_dataframe(cls, tallies):
        return pd.DataFrame([t.to_dict() for t in tallies]).set_index("tally_id")
    
    @classmethod
    def to_json(cls, tallies):
        return cls.to_dataframe(tallies).to_json(orient="records")
    
    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))
    
    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, tallies, filename):
        cls.to_dataframe(tallies).to_csv(filename)


In [ ]:
class sim():
    def __init__(self, title, nps=1e5, prdmp=None, seed=1337, source=None, tallies=None, surfaces=None, cells=None, materials=None):
        self.title = title
        self.nps = nps
        self.prdmp = prdmp
        self.seed = seed


        # stuff thats already an object
        self.source = source
        self.tallies = tallies
        self.surfaces = surfaces
        self.cells = cells
        self.materials = materials
    def __str__(self):
        return f"sim {self.title}"
    def __getitem__(self, key):
        return self.tallies[key]
    def __setitem__(self, key, value):
        self.tallies[key] = value
    def __iter__(self):
        return iter(self.tallies)
    def __len__(self):
        return len(self.tallies)
    def __contains__(self, item):
        return item in self.tallies
    def __eq__(self, other):
        return self.title == other.title
    def __ne__(self, other):
        return self.title != other.title
    def __lt__(self, other):
        return self.title < other.title
    def __le__(self, other):
        return self.title <= other.title
    def __gt__(self, other):
        return self.title > other.title
    def __ge__(self, other):
        return self.title >= other.title
    def __hash__(self):
        return hash(self.title)
    
    @classmethod
    def from_dataframe(cls, df):
        return [cls.from_series(df.loc[i]) for i in df.index]
    
    @classmethod
    def to_dataframe(cls, miscs):
        return pd.DataFrame([m.to_dict() for m in miscs]).set_index("title")
    
    @classmethod
    def to_json(cls, miscs):
        return cls.to_dataframe(miscs).to_json(orient="records")

    @classmethod
    def from_json(cls, json_data):
        return cls.from_dataframe(pd.read_json(json_data))

    @classmethod
    def from_csv(cls, filename):
        return cls.from_dataframe(pd.read_csv(filename))
    
    @classmethod
    def to_csv(cls, miscs, filename):
        cls.to_dataframe(miscs).to_csv(filename)